In [298]:
import pandas as pd
import numpy as np
import scipy.stats as sps
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl
from sklearn import preprocessing
from math import isnan
import datetime as dt
from datetime import datetime
from datetime import timedelta

In [299]:
file = 'cleanDataFinal-yifeng.csv'
myData = pd.read_csv(file)

# Delete unrelated attribute

In [300]:
myData.head(5)

,Unnamed: 0,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
0,0,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
1,1,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0
2,2,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0
3,3,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0
4,4,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0


In [301]:
del_attribute = 'Unnamed: 0'

In [302]:
myData.drop([del_attribute],axis = 1, inplace = True)

In [303]:
myData.head(5)

,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
0,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
1,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0
2,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0
3,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0
4,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0


# Differenciate Date

In [304]:
def str_to_datetime (x):
    return datetime.strptime(x, '%Y-%m-%d')

In [305]:
myData['Date'] = myData['Date'].apply(str_to_datetime)

In [306]:
temp_date = myData.iloc[0,1]
temp_delta = dt.timedelta(0,0,0)

In [307]:
def sameDay_plus1sec (x):
    global temp_date 
    global temp_delta
    if x['Date'] == temp_date:
        temp_delta = temp_delta + dt.timedelta(0,1,0)
        return (temp_delta + x['Date'])
    else:
        temp_date = x['Date']
        temp_delta = dt.timedelta(0,1,0)
        return (temp_delta + x['Date'])

In [308]:
myData['Date'] = myData.apply(sameDay_plus1sec, axis = 1)

In [309]:
myData.set_index('Date', inplace= True)

In [310]:
myData.head(2)

,Cardnum,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
Date,,,,,,,,
2010-01-01 00:00:01,5142190439,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
2010-01-01 00:00:02,5142183973,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0


# Begin Feature Engineering

In [159]:
myData.iloc[6666,]

Cardnum                            5142189945
Merchnum                        2094206450006
Merch description    VWR SCIENTIFIC PROD VCTS
Merch state                                PA
Merch zip                               19380
Transtype                                   P
Amount                                 576.61
Fraud                                       0
Name: 2010-01-31 00:02:09, dtype: object

In [188]:
myData.head(4)

,Cardnum,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
Date,,,,,,,,
2010-01-01 00:00:01,5142190439,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
2010-01-01 00:00:02,5142183973,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0
2010-01-01 00:00:03,5142131721,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0
2010-01-01 00:00:04,5142148452,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0


## 0 days: all attributes

In [364]:
def allAttributes_0 (temp_datetime):
  
    temp_year = temp_datetime.year
    temp_month = temp_datetime.month
    temp_day = temp_datetime.day
    temp_hour = temp_datetime.hour
    temp_minute = temp_datetime.minute
    temp_second = temp_datetime.second
    
    temp_begin = str(temp_month) + '/' + str(temp_day) + '/' + str(temp_year)  \
            +' ' +'00:00'

    temp_end = str(temp_month) + '/' + str(temp_day) + '/' + str(temp_year)  \
            +' ' + str(temp_hour) + ':' + str(temp_minute) + ":" + str(temp_second)
    
    temp0_df = myData.loc[temp_begin:temp_end].copy()
    
    #---------------------------------new attributes for this card------------------------------------------
    
    groupCardnum = temp0_df.groupby(['Cardnum'])
    temp_instance_Cardnum = myData.loc[temp_datetime, 'Cardnum']
    
    '''
    groupCardnum = temp0_df[['Cardnum','Amount']].groupby(['Cardnum']).mean().copy()
    temp_instance_Cardnum = myData.loc[temp_datetime, 'Cardnum']
    myData.loc[temp_datetime, 'Average/Card-0days'] = groupCardnum.loc[temp_instance_Cardnum, 'Amount']
    '''
    
    temp_df = groupCardnum.max().copy()
    
    #print(temp_df.loc[temp_instance_Cardnum, 'Amount'])
    
    myData.loc[temp_datetime, 'Count-0days'] = temp_df.loc[temp_instance_Cardnum, 'Amount']

    #---------------------------------new attributes for this card------------------------------------------

 
   

0:03:33.169348

0:04:14.908239

In [365]:
start_time = dt.datetime.now()
test_count = 1
for i in myData.index:
    test_count = test_count + 1
    if test_count > 1000:
        break
    allAttributes_0(i)
end_time = dt.datetime.now()
print(end_time-start_time)

0:01:44.041192


In [ ]:
0:01:44.041192 max

In [ ]:
0:01:35.336651 min

In [ ]:
0:00:03.050314 mean

In [ ]:
0:00:02.898242 count

In [ ]:
0:00:03.136647 sum 

In [347]:
myData.T

Date,2010-01-01 00:00:01,2010-01-01 00:00:02,2010-01-01 00:00:03,2010-01-01 00:00:04,2010-01-01 00:00:05,2010-01-01 00:00:06,2010-01-01 00:00:07,2010-01-01 00:00:08,2010-01-01 00:00:09,2010-01-01 00:00:10,...,2010-12-31 00:00:43,2010-12-31 00:00:44,2010-12-31 00:00:45,2010-12-31 00:00:46,2010-12-31 00:00:47,2010-12-31 00:00:48,2010-12-31 00:00:49,2010-12-31 00:00:50,2010-12-31 00:00:51,2010-12-31 00:00:52
Cardnum,5142190439,5142183973,5142131721,5142148452,5142190439,5142149874,5142189277,5142191182,5142258629,5142190439,...,5142187448,5142160778,5142226979,5142196337,5142221253,5142276053,5142225701,5142226486,5142244619,5142243247
Merchnum,5509006296254,61003026333,4503082993600,5509006296254,5509006296254,5509006296254,5509006296254,6098208200062,602608969534,5509006296254,...,5725000466504,08-0616075333,602608969534,467619770330,5725000466504,3500000006160,8090710030950,4503057341100,8834000695412,9108347680006
Merch description,FEDEX SHP 12/23/09 AB#,SERVICE MERCHANDISE #81,OFFICE DEPOT #191,FEDEX SHP 12/28/09 AB#,FEDEX SHP 12/23/09 AB#,FEDEX SHP 12/22/09 AB#,FEDEX SHP 12/28/09 AB#,MIAMI COMPUTER SUPPLY,FISHER SCI ATL,FEDEX SHP 12/23/09 AB#,...,CDW*GOVERNMENT INC,COMPUTER STORE OF CORVLS,FISHER SCI ATL,STAY ONLINE,CDW*GOVERNMENT INC,BEST BUY 00001610,MARKUS OFFICE SUPPLIES,"TECH PAC, INC",BUY.COM,STAPLES NATIONAL #471
Merch state,TN,MA,MD,TN,TN,TN,TN,OH,GA,TN,...,IL,OR,GA,NC,IL,KY,OH,OH,CA,NJ
Merch zip,38118,1803,20706,38118,38118,38118,38118,45429,30091,38118,...,60061,97330,30091,27560,60061,41042,45248,45150,92656,7606
Transtype,P,P,P,P,P,P,P,P,P,P,...,P,P,P,P,P,P,P,P,P,P
Amount,3.62,31.42,178.49,3.62,3.62,3.67,3.62,230.32,62.11,3.62,...,2231.08,243.15,32.04,147.35,425,84.79,118.75,363.56,2202.03,554.64
Fraud,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
Average/Card-0days,3.62,31.42,178.49,3.62,3.62,3.67,3.62,230.32,62.11,3.62,...,2231.08,132.955,98.52,147.35,1022.18,84.79,401.875,363.56,2202.03,554.64
Max/Card-0days,3.62,31.42,178.49,3.62,3.62,3.67,3.62,230.32,62.11,3.62,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# maybe it is the time to choose another approach 

# too many resoures are wasted if we do index by index

# on the other hand, what if we first choose to subset the whole dataframe

# all the records for a same card is selected

In [328]:
test1 = '2010-09-20 00:07:10'

In [329]:
test_instance = datetime.strptime(test1, '%Y-%m-%d %H:%M:%S')
test_instance

datetime.datetime(2010, 9, 20, 0, 7, 10)

In [330]:

temp_datetime = test_instance
temp_year = temp_datetime.year
temp_month = temp_datetime.month
temp_day = temp_datetime.day
temp_hour = temp_datetime.hour
temp_minute = temp_datetime.minute
temp_second = temp_datetime.second

temp_begin = str(temp_month) + '/' + str(temp_day) + '/' + str(temp_year)  \
        +' ' +'00:00'

temp_end = str(temp_month) + '/' + str(temp_day) + '/' + str(temp_year)  \
        +' ' + str(temp_hour) + ':' + str(temp_minute) + ":" + str(temp_second)

temp0_df = myData.loc[temp_begin:temp_end].copy()

#---------------------------------new attributes for this card------------------------------------------
groupCardnum = temp0_df[['Cardnum','Amount']].groupby(['Cardnum']).mean().copy()
temp_instance_Cardnum = myData.loc[temp_datetime, 'Cardnum']
myData.loc[temp_datetime, 'Average/Card-0days'] = groupCardnum.loc[temp_instance_Cardnum, 'Amount']

groupCardnum = temp0_df.groupby(['Cardnum']).max().copy()
temp_instance_Cardnum = myData.loc[temp_datetime, 'Cardnum']
myData.loc[temp_datetime, 'Max/Card-0days'] = groupCardnum.loc[temp_instance_Cardnum, 'Amount']

#---------------------------------new attributes for this card------------------------------------------

In [332]:
temp0_df.loc[test_instance]

Cardnum                           5142152613
Merchnum                       4353000719908
Merch description     AMAZON.COM  *SUPERSTOR
Merch state                               WA
Merch zip                              98101
Transtype                                  P
Amount                                335.83
Fraud                                      0
Average/Card-0days                    187.31
Max/Card-0days                        335.83
Name: 2010-09-20 00:07:10, dtype: object

In [290]:
temp0_df[temp0_df['Cardnum'] ==  5142152613]

,Cardnum,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud,Average/Card-0days
Date,,,,,,,,,
2010-09-20 00:00:36,5142152613,6880062529835,MILLER'S OFFICE PRODUCTS,VA,22150.0,P,12.20,0,NaN
2010-09-20 00:04:25,5142152613,9108234610000,STAPLES #941,PA,17201.0,P,213.90,0,NaN
2010-09-20 00:07:10,5142152613,4353000719908,AMAZON.COM *SUPERSTOR,WA,98101.0,P,335.83,0,NaN


In [291]:
groupCardnum.head(5)

,Amount
Cardnum,
5142111125,798.000
5142111572,74.985
5142112611,322.200
5142112858,5443.300
5142112936,270.690


In [295]:
myData.loc[test_instance]

Cardnum                           5142152613
Merchnum                       4353000719908
Merch description     AMAZON.COM  *SUPERSTOR
Merch state                               WA
Merch zip                              98101
Transtype                                  P
Amount                                335.83
Fraud                                      0
Average/Card-0days                    187.31
Name: 2010-09-20 00:07:10, dtype: object

In [275]:
for i in myData.index:
    if myData.loc[i,'Cardnum'] == 5142152613:
        
        print(i)

2010-01-04 00:03:19
2010-01-06 00:03:10
2010-01-07 00:04:05
2010-01-08 00:00:49
2010-01-10 00:01:55
2010-01-10 00:03:45
2010-01-13 00:03:08
2010-01-14 00:01:15
2010-01-15 00:00:40
2010-01-17 00:02:44
2010-01-25 00:04:03
2010-01-31 00:00:42
2010-02-02 00:01:12
2010-02-03 00:02:46
2010-02-09 00:03:15
2010-02-14 00:02:15
2010-02-15 00:02:24
2010-02-15 00:05:04
2010-02-15 00:05:35
2010-02-16 00:06:55
2010-02-17 00:00:19
2010-02-22 00:03:34
2010-02-23 00:04:48
2010-02-24 00:00:51
2010-02-24 00:03:11
2010-02-26 00:00:09
2010-02-26 00:01:16
2010-02-28 00:07:55
2010-02-28 00:08:50
2010-03-01 00:04:40
2010-03-06 00:00:04
2010-03-06 00:03:19
2010-03-07 00:01:03
2010-03-08 00:03:29
2010-03-08 00:04:23
2010-03-14 00:04:55
2010-03-16 00:04:35
2010-03-18 00:01:55
2010-03-18 00:01:57
2010-03-20 00:03:26
2010-03-24 00:04:01
2010-03-28 00:01:12
2010-03-29 00:02:19
2010-03-29 00:02:32
2010-03-29 00:04:45
2010-03-29 00:04:56
2010-03-30 00:03:04
2010-03-31 00:03:13
2010-04-03 00:01:28
2010-04-03 00:02:16


In [88]:
myData.to_csv('featureEngineering_yifeng.csv')